In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [30]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)  # 1 channel (grayscale)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)  # 16*5*5 is flattened conv2 output
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 output classes for digits 0-9

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 16*5*5)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [32]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Grayscale(),  # Convert to grayscale
    transforms.Resize((32, 32)),  # Resize to 32x32
    transforms.ToTensor()  # Convert to PyTorch tensor
])


In [34]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Load the dataset
train_data = ImageFolder(root=r'C:\\Users\\swaya\Desktop\\Hand_Written', transform=transform)
test_data = ImageFolder(root=r'C:\\Users\\swaya\\Desktop\\Hand_Written', transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [36]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")


Epoch [1/10], Loss: 2.238250970840454
Epoch [2/10], Loss: 2.2093567848205566
Epoch [3/10], Loss: 2.1767418384552
Epoch [4/10], Loss: 2.1419761180877686
Epoch [5/10], Loss: 2.100572109222412
Epoch [6/10], Loss: 2.0454823970794678
Epoch [7/10], Loss: 1.9768003225326538
Epoch [8/10], Loss: 1.8892549276351929
Epoch [9/10], Loss: 1.778083086013794
Epoch [10/10], Loss: 1.6382476091384888


In [38]:
model.eval()
digit_counts = [0] * 10  # Initialize a list to count predictions for digits 0-9

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        # Count predictions for each digit
        for prediction in predicted:
            digit_counts[prediction.item()] += 1

# Display the results
for digit, count in enumerate(digit_counts):
    print(f"Digit {digit}: {count} predictions")



Digit 0: 2 predictions
Digit 1: 0 predictions
Digit 2: 0 predictions
Digit 3: 0 predictions
Digit 4: 0 predictions
Digit 5: 0 predictions
Digit 6: 0 predictions
Digit 7: 0 predictions
Digit 8: 0 predictions
Digit 9: 0 predictions
